For this tutorial the following packages are required in addition to the `simframe` requirements:

* `scipy`

In this tutorial you'll learn about the basic usage of `simframe`.

# Simple Integration

This lesson is a showcase for setting up a simple integration with `simframe`. And explains how to write/read data files.

The goal is to integrate the following differential equation

$\frac{\mathrm{d}Y}{\mathrm{d}x} = b Y$

with the initial condition

$Y \left( 0 \right) = A$.

The solution of this problem is given by

$Y \left( x \right) = A \cdot \exp \left( bx \right)$.

## Setting the problem parameters

In [ ]:
A  = 10. # Initial value
b  = -1. # decay scale
dx = 0.1 # Stepsize

## Initializing the simulation frame

In [2]:
from simframe import Frame

In [3]:
sim = Frame(description="Simple Integration")

In [4]:
sim

Frame (Simple Integration)
--------------------------
    Integrator   : not specified
    Writer       : not specified

This created an empty framework `sim`.

As you can see the Integrator and Writer of the Frame object are not set initially. The Writer contains the instructions for writing output files. If you do not want to outputs you can ommit setting it.

The Integrator, on the other hand, has to be set for a successful execution. At the minimum it has to control when to stop the simulation.

## Setting the Writer

`simframe` comes with a built-in writer for writing outputs in the `.hdf5` file format using the `h5py` module.

In [5]:
from simframe import writers

In [6]:
writers.hdf5writer

Writer (HDF5 file format using h5py)
------------------------------------
    Data directory : data
    File names     : data/data0000.hdf5
    Overwrite      : False
    options        : {'com': 'lzf', 'comopts': None}

We want to change the name of the output directory and we want to set the writer to overwrite existing files. By default the writer will raise an exception if the file it needs to write already exists to protect your data.

In [7]:
writers.hdf5writer.datadir = "1_data"
writers.hdf5writer.overwrite = True

In [8]:
writers.hdf5writer

Writer (HDF5 file format using h5py)
------------------------------------
    Data directory : 1_data
    File names     : 1_data/data0000.hdf5
    Overwrite      : True
    options        : {'com': 'lzf', 'comopts': None}

The writer can be simply assigned to the frame object

In [9]:
sim.writer = writers.hdf5writer

In [10]:
sim

Frame (Simple Integration)
--------------------------
    Integrator   : not specified
    Writer       : Writer (HDF5 file format using h5py)

## Adding fields

We now have to add fields to our simulation frame, which has a method for this task.

Here we add the field for our variable `Y` and initialize it with the initial condition `A`.\
The value you set has to be of the correct shape and data type. It cannot be changed later.

In [11]:
sim.addfield("Y", A)

In [12]:
sim

Frame (Simple Integration)
--------------------------
    Y            : Field
  -----
    Integrator   : not specified
    Writer       : Writer (HDF5 file format using h5py)

The field `Y` can be easily accessed and manipulated. All `NumPy` functions for `ndarray` work on it.

For example:

In [13]:
sim.Y

10.0

In [14]:
sim.Y * 2

20.0

In [15]:
import numpy as np

np.exp(sim.Y)

22026.465794806718

We now have to add a field for `x`. But since this is our integration variable, we have to add a special field for it.\
Here we set the initial value to `0`:

In [16]:
sim.addintegrationvariable("x", 0.)

In [17]:
sim

Frame (Simple Integration)
--------------------------
    x            : Field, Integration variable
    Y            : Field
  -----
    Integrator   : not specified
    Writer       : Writer (HDF5 file format using h5py)

An integration variable is a special field that has additional functionality for the stepsize of our simulation.

## Adding functions

First we define a function that controls the stepsize of the simulation. It needs to have the frame object as argument. Therefore, anything within the object can be accessed. The function needs to return the value of the stepsize

In this simple integration, we want to have a constant stepsize, that we defined above.

In [18]:
def fdx(sim):
    return dx

We can now simply assign this function to the integration variable

In [19]:
sim.x.updater = fdx

Furthermore, the integration variable needs to know the desired snapshots. Snapshots define the value of the integration variable at which an output should be written. Even if you do not want to write output files the list of snapshots needs to contain at least one value, which defines when the simulation should be stopped.

Here we want to have 20 output files written between `x=0.5` and `x=10.` and assign them to the integration variable.

In [20]:
snaps = np.linspace(0.5, 10., 20)
sim.x.snapshots = snaps

The integration variable has now anything it needs to access its additional functionality which are the stepsize,

In [21]:
sim.x.updater.updater.update(sim)

0.1

In [22]:
sim.x.stepsize

0.1

the maximum possible stepsize, i.e., the stepsize until the next snapshot,

In [23]:
sim.x.maxstepsize

0.5

and the value of the next snapshot, which is here identical to the maximum stepsize.

In [24]:
sim.x.nextsnapshot

0.5

## Setting the differential equation

We also have to define the differential equation of our variable `Y`, that we want to integrate. The function needs as argument the frame object and the variable itself.

In [25]:
def dYdx(sim, Y):
    return b*Y

We can simply assign it to the variable as before.

In [26]:
sim.Y.differentiator = dYdx

## Setting the integrator

Last but not least we have to specify the integrator which controls the simulation. The integrator needs the integration variable as argument upon initialization.

In [27]:
from simframe import Integrator

sim.integrator = Integrator(sim.x, description="Simple first order integration")

In [28]:
sim

Frame (Simple Integration)
--------------------------
    x            : Field, Integration variable
    Y            : Field
  -----
    Integrator   : Integrator (Simple first order integration)
    Writer       : Writer (HDF5 file format using h5py)

If we ran the simulation now, the output files would be written, but `Y` would stay constant. The reason is that the integrator needs a set of instructions that specifiy what to do.

An instruction specifies the integration scheme, the variable to be integrated, and the fraction of the stepsize it should operate on. `simframe` provides a simple Euler 1st order scheme. We'll explain later how to create your own schemes.

In [29]:
from simframe import Instruction
from simframe import schemes

Here we set up our instruction for integrating `Y` with a simple Euler 1st order scheme by using the full timestep.

In [30]:
instructions = [Instruction(schemes.expl_1_euler, sim.Y)]

We simply add this list to our integrator.

In [31]:
sim.integrator.instructions = instructions

## Running the simulation

The simulation is now ready to go!

In [32]:
sim.run()

Writing file '1_data/data0000.hdf5'
Writing file '1_data/data0001.hdf5'
Writing file '1_data/data0002.hdf5'
Writing file '1_data/data0003.hdf5'
Writing file '1_data/data0004.hdf5'
Writing file '1_data/data0005.hdf5'
Writing file '1_data/data0006.hdf5'
Writing file '1_data/data0007.hdf5'
Writing file '1_data/data0008.hdf5'
Writing file '1_data/data0009.hdf5'
Writing file '1_data/data0010.hdf5'
Writing file '1_data/data0011.hdf5'
Writing file '1_data/data0012.hdf5'
Writing file '1_data/data0013.hdf5'
Writing file '1_data/data0014.hdf5'
Writing file '1_data/data0015.hdf5'
Writing file '1_data/data0016.hdf5'
Writing file '1_data/data0017.hdf5'
Writing file '1_data/data0018.hdf5'
Writing file '1_data/data0019.hdf5'
Writing file '1_data/data0020.hdf5'


Since the data directory did not exist before, it was created by `simframe` and the output files have been written.

## Reading the outputs

Every writer should provide a reader for its specific file format. It can either list all data files in the data directory,

In [33]:
sim.writer.read.listfiles()

['1_data/data0000.hdf5',
 '1_data/data0001.hdf5',
 '1_data/data0002.hdf5',
 '1_data/data0003.hdf5',
 '1_data/data0004.hdf5',
 '1_data/data0005.hdf5',
 '1_data/data0006.hdf5',
 '1_data/data0007.hdf5',
 '1_data/data0008.hdf5',
 '1_data/data0009.hdf5',
 '1_data/data0010.hdf5',
 '1_data/data0011.hdf5',
 '1_data/data0012.hdf5',
 '1_data/data0013.hdf5',
 '1_data/data0014.hdf5',
 '1_data/data0015.hdf5',
 '1_data/data0016.hdf5',
 '1_data/data0017.hdf5',
 '1_data/data0018.hdf5',
 '1_data/data0019.hdf5',
 '1_data/data0020.hdf5']

read a single output file,

In [34]:
output3 = sim.writer.read.output(3)

or read the full data set.

In [35]:
data = sim.writer.read.all()

The data is easily accesible from the framework namespace, e.g.

In [36]:
data.x

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
        5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ])

## Fitting of the data

This is a simple fit of the data to see if the original values of `A` and `b` can be obtained.

In [37]:
def fitfunc(x, A, b):
    return A*np.exp(b*x)

In [38]:
from scipy.optimize import curve_fit

popt, pcov = curve_fit(fitfunc, data.x, data.Y)

In [39]:
from IPython.display import Markdown as md
md("| |Simulation|Exact|\n|-|-|-|\n|A|{:4.2f}|{:4.2f}|\n|b|{:4.2f}|{:4.2f}|".format(popt[0],A,popt[1],b))

| |Simulation|Exact|
|-|-|-|
|A|10.00|10.00|
|b|-1.05|-1.00|

You can re-run the simulation and play around with `A`, `b`, and `dx` to see how this simple integration behaves. Note: `dx` will never be larger than the distance to the next snapshot. If you want to increase `dx` beyond that you have to reduce the number of snapshots.